# Making predictions using our trained model

In [1]:
from transformers import BertModel, BertTokenizer, BertConfig
import torch
from torch import nn

In [4]:
class SentimentClassifierModel(nn.Module):

    def __init__(self, freeze_bert = True):
        super(SentimentClassifierModel, self).__init__()

        # Instantiating the BERT model object 
        self.bert_layer = BertModel(BertConfig())
        
        # Defining layers like dropout and linear
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, 1)

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        # Getting hidden states from BERT Layer
        hidden_states = self.bert_layer(seq, attention_mask = attn_masks).last_hidden_state

        # Obtaining the representation of [CLS] head
        cls_rep = hidden_states[:, 0]
        # print('CLS shape: ',cls_rep.shape)

        # Feeding cls_rep to the classifier layer
        logits = self.classifier(cls_rep)
        # print('Logits shape: ',logits.shape)

        return logits

In [7]:
def preprocess(sentence, maxlen=64):

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Tokenize the sentence
    tokens = tokenizer.tokenize(sentence)

    # Inserting the CLS and SEP token at the beginning and end of the sentence
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    
    # Padding/truncating the sentences to the maximum length
    if len(tokens) < maxlen:
        tokens = tokens + ['[PAD]' for _ in range(maxlen - len(tokens))]
    else:
        tokens = tokens[:maxlen-1] + ['[SEP]']
    
    # Convert the sequence to ids with BERT Vocabulary
    tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
    
    # Converting the list to a pytorch tensor
    tokens_ids_tensor = torch.tensor(tokens_ids).unsqueeze(0)

    # Obtaining the attention mask
    attn_mask = (tokens_ids_tensor != 0).long()

    return tokens_ids_tensor, attn_mask

In [13]:
# Defining an evaluation function for training 
def predict(net, iseq, masks):
    device = 'cpu'
    # Setting model to evaluation mode
    net.eval()

    # Move inputs and targets to device
    iseq, masks = iseq.to(device), masks.to(device)

    # Get logit predictions
    p_logit = net(iseq, masks)

    probs = torch.sigmoid(p_logit.unsqueeze(-1))
    preds = (probs > 0.5).long().squeeze(0)

   
    return preds.item(), probs.item()

In [6]:
# initializing the predictor
predictor = SentimentClassifierModel()
predictor.load_state_dict(torch.load('inference.pth'))

<All keys matched successfully>

In [20]:
# preprocessing a test sentence
test_tokens, test_attn = preprocess('the literally hate this movie ever')

In [21]:
# making predictions of the test sentence
pred, probability = predict(predictor, test_tokens, test_attn)
print(pred, probability)

0 0.04564647749066353


# Testing our Inference Module

In [23]:
from inference import SentimentClassifier, Predictor
import torch

In [24]:
model = SentimentClassifier()
model.load_state_dict(torch.load('inference.pth'))

<All keys matched successfully>

In [25]:
predictor_obj = Predictor(model)

In [26]:
predictor_obj.predict('the literally love this movie ever')

(1, 0.9788225293159485)